In [1]:
import jax.numpy as jnp
import pymc3 as pm3
from include.init import ModelInitializer_2d

RuntimeError: module compiled against API version 0xf but this version of numpy is 0xe

ModuleNotFoundError: No module named 'pymc3'

In [ ]:
number_u = 20
number_f = 10
number_init = 5
number_bound = 4
sample_num = 10
model_initializer = ModelInitializer_2d(number_u=number_u, number_f=number_f, sample_num=sample_num, number_init=number_init, number_bound=number_bound)

In [ ]:
Xu = model_initializer.Xu
yu = model_initializer.yu
xu_noise = model_initializer.xu_noise
tu_noise = model_initializer.tu_noise
Xu_noise = model_initializer.Xu_noise
yu_noise = model_initializer.yu_noise
xu_fixed = model_initializer.xu_fixed
tu_fixed = model_initializer.tu_fixed
Xu_fixed = model_initializer.Xu_fixed
Yu_fixed = model_initializer.Yu_fixed
Y = model_initializer.Y
xf = model_initializer.xf
tf = model_initializer.tf
Xf = model_initializer.Xf
yf = model_initializer.yf
prior_std = 1e1

In [ ]:
from include.heat2d import heat_equation_kuu_noise, heat_equation_kuf, heat_equation_kfu, heat_equation_kff
def compute_K(z_prior, init,  Xfz, Xfg):
    Xuz = z_prior
    params = {'sigma': init[-1][0], 'lengthscale': init[-1][1]}
    zz_uu = heat_equation_kuu_noise(Xuz, Xuz, params)
    zz_uf = heat_equation_kuf(Xuz, Xfz, params)
    zg_uf = heat_equation_kfu(Xuz, Xfg, params)
    zz_fu = heat_equation_kfu(Xfz, Xuz, params)
    zz_ff = heat_equation_kff(Xfz, Xfz, params)
    zg_ff = heat_equation_kff(Xfz, Xfg, params)
    gz_fu = heat_equation_kfu(Xfg, Xuz, params)
    gz_ff = heat_equation_kff(Xfg, Xfz, params)
    gg_ff = heat_equation_kff(Xfg, Xfg, params)
    K = jnp.block([[zz_uu, zz_uf, zg_uf], [zz_fu, zz_ff, zg_ff], [gz_fu, gz_ff, gg_ff]])
    return K

In [ ]:
init_params = model_initializer.heat_params_init
Xfz =  Xu_fixed
Xfg=Xf

In [ ]:
import numpy
prior_mean=Xu_noise 
prior_cov=prior_std *jnp.eye(2)
n_samples = 20
coords = {"samples": range(n_samples),
          "features": ['x', 't']}
basic_model = pm.Model()
prior_mean = numpy.array(prior_mean)
prior_cov = numpy.array(prior_cov)

In [ ]:
n_samples = 20
dimension = 2

with pm.Model() as model:
    z_uncertain = pm.MvNormal('z_uncertain', mu=prior_mean, cov=prior_cov, shape=(n_samples, dimension))
    trace = pm.sample(10)

    print(trace.varnames)


In [ ]:
z_uncertain_samples = trace['z_uncertain']
z_uncertain_samples

In [ ]:
K = compute_K(z_uncertain, init_params, Xfz, Xfg)